In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import nltk
nltk.download("gutenberg")
from nltk.corpus import gutenberg
import pandas as pd 
import mlflow
import os
# os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"
# mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
# try:
#     mlflow.set_experiment("lstm_gru")
# except mlflow.exceptions.MlflowException:
#     mlflow.create_experiment("lstm_gru")
#     mlflow.set_experiment("lstm_gru")

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/sarthakagarwal/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
# data = gutenberg.raw('shakespeare-hamlet.txt')

In [3]:
# with open("data/hamlet.txt", "w") as file:
#     file.write(data)

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [6]:
with open("data/hamlet.txt", "r") as file:
    text = file.read().lower()

In [7]:
# Index creation for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
print(total_words)


4818


In [8]:
# input sequence creation
input_sequence = []

for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequence.append(n_gram_sequence)
    

In [9]:
# pad sequence 
max_sequence_length = max(len(i) for i in input_sequence)
input_sequence = np.array(
    pad_sequences(
        input_sequence, 
        maxlen=max_sequence_length,
        padding='pre'
    )
)

In [10]:
# create predictor and labels
import tensorflow as tf 

x, y = input_sequence[:, :-1], input_sequence[:, -1]

In [11]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=42
)

In [ ]:
# train our lstm model on the dataset
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(Bidirectional(GRU(1024)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(512)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(256)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(128)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(64)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(32)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(16)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(8)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(4)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(2)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(1)))
model.add(Dense(total_words, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


# model=Sequential()
# model.add(Embedding(total_words,100,input_length=max_sequence_length-1))
# model.add(GRU(150,return_sequences=True))
# model.add(Dropout(0.2))
# model.add(GRU(100))
# model.add(Dense(total_words,activation="softmax"))

# # #Compile the model
# model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
# model.summary()


callback = EarlyStopping(monitor='val_loss', patience=10,
                         restore_best_weights=True)

2025-04-07 14:36:12.092552: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-07 14:36:12.092764: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-07 14:36:12.092781: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1744016772.093028 91601688 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1744016772.093225 91601688 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# mlflow.autolog()
# with mlflow.start_run(run_name="lstm"):
history = model.fit(
    x_train,
    y_train,
    epochs=1000,
    validation_data=(x_test, y_test),
    verbose=1,
    callbacks=[callback]
)

Epoch 1/100


2025-04-07 14:36:14.373658: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


644/644 ━━━━━━━━━━━━━━━━━━━━ 93s 135ms/step - accuracy: 0.0321 - loss: 7.1604 - val_accuracy: 0.0451 - val_loss: 6.6812
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 79s 123ms/step - accuracy: 0.0500 - loss: 6.3378 - val_accuracy: 0.0633 - val_loss: 6.5668
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 86s 134ms/step - accuracy: 0.0752 - loss: 5.9155 - val_accuracy: 0.0740 - val_loss: 6.6014
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 80s 125ms/step - accuracy: 0.0959 - loss: 5.3986 - val_accuracy: 0.0748 - val_loss: 6.7002
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 80s 124ms/step - accuracy: 0.1202 - loss: 4.8652 - val_accuracy: 0.0694 - val_loss: 6.9392
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 81s 125ms/step - accuracy: 0.1673 - loss: 4.3011 - val_accuracy: 0.0657 - val_loss: 7.1531
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 80s 125ms/step - accuracy: 0.2512 - loss: 3.6869 - val_accuracy: 0.0649 - val_loss: 7.4566
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 82s 127ms/step - accuracy: 0.3433 - loss: 3.134

In [25]:
def predict(model, tokenizer, text, max_sequence_length):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_length:
        token_list = token_list[-max_sequence_length+1:]
    token_list = pad_sequences(
        [token_list],
        maxlen=max_sequence_length-1,
        padding='pre',
    )
    predicted = model.predict(token_list, verbose=0)
    preicted_word_index = np.argmax(predicted, axis = 1)
    for word, index in tokenizer.word_index.items():
        if index == preicted_word_index:
            return word
    
    return None

In [26]:
predict(
    model,
    tokenizer=tokenizer,
    max_sequence_length=max_sequence_length,
    text="to be or not to be",
)

'the'

In [27]:
model.save("model_gru_bidirectional.h5")
import pickle
with open("tokenizer_gru_bidirectional.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 13, 150)        │       113,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 150)            │       135,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4818)           │       727,518 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,375,856 (16.69 MB)

 Trainable params: 1,458,618 (5.56 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,917,238 (11.13 MB)